# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:

orders = pd.read_csv("../datasets/Orders.csv", index_col= 0)
orders_original = orders.copy()

orders.info()
orders.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 13 columns):
InvoiceNo       397924 non-null int64
StockCode       397924 non-null object
year            397924 non-null int64
month           397924 non-null int64
day             397924 non-null int64
hour            397924 non-null int64
Description     397924 non-null object
Quantity        397924 non-null int64
InvoiceDate     397924 non-null object
UnitPrice       397924 non-null float64
CustomerID      397924 non-null int64
Country         397924 non-null object
amount_spent    397924 non-null float64
dtypes: float64(2), int64(7), object(4)
memory usage: 42.5+ MB


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [6]:
# VIP Customers above the 95th percentile
# Preferred Customers 75th and 95th percentile

# Sub Problem 1: How to aggregate the amount_spent for unique customers?

# First CustomerID is unique_customers
len(orders.CustomerID.unique())

4339

In [7]:
# Sub Problem 1: GroupBy CustomerID and then summation
orders.groupby("CustomerID").sum()

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,
12346,541431,2011,1,2,10,74215,1.04,77183.60
12347,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,16869685,62324,257,111,472,2341,178.71,1797.24
12349,42165457,146803,803,73,657,631,605.10,1757.55
12350,9231629,34187,34,51,272,197,65.30,334.40
...,...,...,...,...,...,...,...,...
18280,5457120,20110,30,10,90,45,47.65,180.60
18281,3895248,14077,42,49,70,54,39.36,80.82
18282,6838540,24132,116,60,146,103,62.39,178.05


In [8]:
orders.groupby("CustomerID").sum()['amount_spent']
x= pd.DataFrame(orders.groupby("CustomerID").sum()['amount_spent'])
x

vip_q= x.quantile(0.95)
y = pd.DataFrame(vip_q)
y = 5840

vip=[]
for customer in x:
    for j in x['amount_spent']:
        if j > y:
            print("test")
            

            
pref= x.quantile(0.75)
pref= 1661.64

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


amount_spent    1661.64
Name: 0.75, dtype: float64

In [ ]:
# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"

# Table.with_column(label, values[, formatter])

# Return a new table with an additional or replaced column




Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# groupby country in vip customers


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# groupby country after summing up vip +pref customers

